In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import json, pprint
from pathlib import Path
from pyspark.sql import SparkSession

In [3]:
def try_import(name):
    import sys, os, csv
    from pathlib import Path
    from toolz import concatv
    cwd = Path()
    reader = sorted(concatv(
        [('sys', 'script', sys.argv[0], name)],
        [('sys', 'executable', sys.executable, name)],
        [('path', 'cwd', str(cwd.absolute()), name)],
        sorted(('module', k, str(mod), name) for k, mod in sys.modules.items()),
        sorted(('environ', k, str(val), name) for k, val in dict(os.environ).items()),
    ))
    
    infile = cwd /f"{name}.csv"

    with infile.open('w') as fh:
        writer = csv.writer(fh)
        writer.writerow(['category','type','item', 'source'])
        for rec in reader:
            writer.writerow(rec)
    result = {'infile': str(infile),
            'record_cnt': len(reader)}
    print(f'>> result={result}')
    import _scproxy
    #import numpy
    #import pandas
    return result

In [4]:
ret = try_import('notebook_local')
ret

>> result={'infile': 'notebook_local.csv', 'record_cnt': 972}


{'infile': 'notebook_local.csv', 'record_cnt': 972}

In [5]:
spark = (SparkSession
        .builder
        .master("local")
        .appName("Import Test")
        .getOrCreate() )
df = spark.createDataFrame([{'iteration': 1} ])
rdd = df.rdd.map(lambda x: try_import('notebook_spark'))
ret = rdd.take(1)
pprint.pprint(ret)

/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost, executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$3.applyOrElse(PythonRunner.scala:490)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$3.applyOrElse(PythonRunner.scala:479)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:36)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:597)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:575)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:410)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.s

In [ ]:
import pandas as pd

In [130]:
reader = Path().glob('cmd*.csv')
reader = map(pd.read_csv,reader)
df = pd.concat(reader)
df = df.set_index(['category','type','source']).item.unstack()
df= df.sort_index()
df['match'] = df.fillna('na').cmd_local == df.fillna('na').cmd_spark
df = df[~df.match].copy()
DATA = df

In [197]:

df= DATA.xs('module').copy()
ds = df.cmd_local.str.extract(r"from '(.+)'")
df['cmd_local'] = ds.where(ds.notna(), df.cmd_local, axis=0)

ds = df.cmd_spark.str.extract(r"from '(.+)'")
df['cmd_spark'] = ds.where(ds.notna(), df.cmd_spark, axis=0)

In [198]:
with pd.option_context('max_rows',None, 'max_columns', None, 'max_colwidth', 100):
    display(df)

source,cmd_local,cmd_spark,match
type,,,
__main__,./import_test.py,/Users/wmcabee/opt/spark/python/lib/pyspark.zip/pyspark/daemon.py,False
org,NaN,<module 'org' (namespace)>,False
py4j,/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/py4j/__init__.py,/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/__init__.py,False
py4j.compat,/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/py4j/compat.py,/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/compat.py,False
py4j.finalizer,/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/py4j/finalizer.py,/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/finalizer.py,False
py4j.java_collections,/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/py4j/java_collections.py,/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_collections.py,False
py4j.java_gateway,/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/py4j/java_gateway.py,/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py,False
py4j.protocol,/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/py4j/protocol.py,/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py,False
py4j.signals,/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/py4j/signals.py,/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/signals.py,False
